In [10]:
# # 查看有哪些python库可用
# import pkg_resources
# installed_packages = pkg_resources.working_set
# installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
# print(installed_packages_list)
# # # 查看可用python库的位置
# # import pkg_resources
# # installed_packages = pkg_resources.working_set
# # for package in installed_packages:
# #     print(package.key, ":", package.version)
# #     print("Location:", package.location)


# #清理所有多余文件
# import pandas as pd
# import glob
# files = glob.glob("*.ipynb")
# for file in files:
#     if ("实盘" not in file):
#         remove_file(f"./{file}", trash=False)#删除所有文件
# files = glob.glob("*.csv")
# df=pd.DataFrame({})
# for file in files:
#     if "调整后" in file:
#         thisdf=pd.read_csv(file)
#         thisdf=thisdf[-1:]
#         df=pd.concat([df,thisdf])
#         print(len(df))
#     # if ("行业" not in file):
#     #     remove_file(f"./{file}", trash=False)#删除所有文件
# df.to_csv(f"择时因子.csv")#数据拼接


# #数据压缩并下载
# import glob
# files = glob.glob("*.csv")
# # 将所有文件打包成一个 zip 文件
# import zipfile
# file_size=0
# with zipfile.ZipFile("数据打包.zip", "w") as zipf:
#     for file in files:
#         zipf.write(file)
#         file_size+=zipf.infolist()[-1].file_size
#         if zipf.infolist()[0].file_size >= 49*1024*1024:
#             break
#         print(file,file_size)


# #获取指数数据及成分股
# today=get_trade_days(end_date=start_date,count=2).values[1]
# today=np.datetime_as_string(today,unit="D").replace("-","")
# yesterday=get_trade_days(end_date=start_date,count=2).values[0]
# preyesterday=np.datetime_as_string(yesterday,unit="D")
# yesterday=preyesterday.replace("-","")
# print("******","等待策略初始化",today,yesterday)
# df=get_all_securities(ty=None,date=None)
# df=df.reset_index(drop=False)
# # print(df)
# # df.to_csv("df.csv")
# # thisdf=df[df["exchange"]=="北交所"]
# thisdf=df[df["display_name"]=="北证50"]
# print(thisdf)
# thissymbol=thisdf["symbol"].values[0]
# print(thissymbol)
# df=get_index_weight(thissymbol,yesterday)
# print("北证50昨日权重",df)


# # yesterday ln数据标准化
# def signlog(X):
#     #np.sign((X)：返回 X 的符号。如果 X 是正数，返回1；如果 X 是负数，返回-1；如果 X 是0，返回0。
#     #np.log(1.0 + abs(X))：计算 1.0 + abs(X) 的自然对数，其中 np 通常指的是 Python 中的 NumPy 库，np.log 表示自然对数。
#     # factor value因子值
#     return np.sign(X)*np.log(1.0 + abs(X))

# 过滤科创北交股票
def filter_kcb_stock(stocks): 
    for stock in stocks[:]:
        if stock[0]=="4" or stock[0]=="8" or stock[:2]=="68":
            stocks.remove(stock)
        # if stock[0]=="4" or stock[0]=="8" or stock[:2]=="68" or stock[:2]=="30":
        #     stocks.remove(stock)
    return stocks

#清空所有板块
from sklearn.svm import SVR
import pandas as pd
bk=custom_sector() # 返回所有板块的dataframe
print(bk)
if len(bk)>0:
    for bkname in bk["name"].tolist():
        # if bkname not in targetlist:
            # bkstocks=custom_sector(bkname) # 返回板块的属性和股票
            # print("返回板块内的标的",bkstocks)
            re=custom_sector(bkname,'remove') # 删除无用板块
            print("删除板块返回值",re)
# #根据具体文件插入板块
# df=pd.read_csv("printdf截取后.csv")
# dfgroup=df.groupby("floattoday")
# for index,thisdf in dfgroup:
#     print(index)
#     re=custom_sector(str(int(index)), 'insert', thisdf["代码"].tolist())#新建一个板块
#     # custom_sector('板块1', 'update', ['000001.SZ'], '新名字') # 更新板块1的股票列表或名字
#     # custom_sector('板块1', 'append', ['000001.SZ']) # 增加板块1的股票列表
#     # custom_sector('板块1', 'pop', ['000001.SZ']) # 移除板块1的股票
#     print(re)

#日期选股
numbuystock=30#取30只股票
import pandas as pd
import numpy as np
import datetime
import math
daysnum=1800#获取daysnum天之前的数据
# daysnum=5#获取daysnum天之前的数据
start_date=((datetime.datetime.now()-datetime.timedelta(days=daysnum)).strftime("%Y-%m-%d")) #测试当天的数据
print(start_date)
end_date=((datetime.datetime.now()).strftime("%Y-%m-%d")) #测试当天的数据
tradeday=get_trade_days(start_date=start_date,end_date=end_date).values
print(len(tradeday))
tradeday=[np.datetime_as_string(x,unit="D").replace("-","") for x in tradeday]
print("start_date",start_date,"tradeday",tradeday,len(tradeday),type(tradeday),type(tradeday[0]))

targetlist=["市值"]
# targetlist=["趋势","市值","估值"]
for target in targetlist:
    for today in tradeday:
        index=tradeday.index(today)
        print("当前的选股日期",tradeday[index])
        if (index>0):
            yesterday=tradeday[index-1]
            if target=="市值":
                #每日选股【小市值】
                stocks=get_index_stocks("000985.CSI",yesterday)#中证全指
                stocks=filter_kcb_stock(stocks) # 去除科创北交
                print(len(stocks))
                df=get_price(
                    securities=stocks,
                    start_date=None,
                    end_date=today,
                    fre_step='1d',
                    fields=["open","is_paused","is_st","prev_close","high_limit","low_limit"], # 获取全部数据列，其中昨日收盘价是用来计算总股本的
                    # fq='pre',
                    fq=None, # 这里动态复权或者不复权都是可以的
                    bar_count=1,
                    is_panel=1).to_frame()
                df=df.reset_index()
                df=df[(df["is_paused"]==0)&(df["is_st"]==0)]
                df=df[df["open"]>4]
                df=df.rename(columns={"minor": "代码","high_limit":"涨停价","low_limit":"跌停价"})
                df=df[["代码","涨停价","跌停价"]]
                stocks=df["代码"].tolist()
                # 获取基本面数据
                olddf=get_fundamentals(query(
                        asharevalue.symbol,#代码
                        asharevalue.total_shares,#总股本
                        asharevalue.total_mv,#总市值
                        income.profit_before_tax,#利润总额【当计提历史亏损的时候利润总额大于当年净利润】
                        income.net_profit,#净利润
                        income.np_atsopc,#归母净利润
                        balance.undistributed_profits,#未分配利润
                        income.basic_eps,#基本每股收益（真实的，稀释每股收益是预期的）
                        balance.total_liabilities,#负债合计
                        balance.total_liab_and_holders_equity,#负债和股东权益总计
                    ),date=yesterday)
                olddf=olddf.rename(columns={
                        "asharevalue_symbol":"代码",
                        "asharevalue_total_shares":"总股本",
                        "asharevalue_total_mv":"总市值",
                        "income_profit_before_tax":"利润总额",
                        "income_net_profit":"净利润",
                        "income_np_atsopc":"归母净利润",
                        "income_basic_eps":"基本每股收益",
                        "balance_total_liabilities":"负债合计",
                        "balance_total_liab_and_holders_equity":"负债和股东权益总计",
                        "balance_undistributed_profits":"未分配利润",
                        })
                olddf["资产负债率"]=olddf["负债合计"]/olddf["负债和股东权益总计"]
                olddf=olddf[olddf["资产负债率"]<1]#存在资产负债率大于1（资不抵债）的标的，跟历史数据一致
                olddf=olddf[olddf["利润总额"]>0]
                olddf=olddf[olddf["净利润"]>0]
                olddf=olddf[olddf["归母净利润"]>0]
                olddf=olddf[olddf["未分配利润"]>0]
                olddf["总市值rank"]=olddf.sort_values(by='总市值',ascending=True)["总市值"].rank(method='min')#从小到大排序
                olddf["排名"]=olddf["总市值rank"]
                # olddf.to_csv("双排名.csv")
                dftwo=olddf.nsmallest(math.ceil(1.2*numbuystock),"排名")
                dfone=olddf.nsmallest(math.ceil(numbuystock),"排名")
                dftwo=dftwo.sort_values(by='总市值',ascending=True)#ascending=True从小到大排序
                dfone=dfone.sort_values(by='总市值',ascending=True)#ascending=True从小到大排序
                name="总市值"
                try:
                    re=custom_sector(str(int(tradeday[index]))+name,'insert',dfone["代码"].tolist())#新建一个板块【直接插入容易导致顺序不对】
                    print(re)
                except:
                    re=custom_sector(str(int(tradeday[index]))+name,'remove') # 删除无用板块
                    print("删除板块返回值",re)
                    re=custom_sector(str(int(tradeday[index]))+name,'insert',dfone["代码"].tolist())#新建一个板块【直接插入容易导致顺序不对】
                    print(re)
            if target=="估值":
                #每日选股【基本面】
                stocks=get_index_stocks("000985.CSI",yesterday)#中证全指
                stocks=filter_kcb_stock(stocks) # 去除科创北交
                #过滤ST停牌
                df=get_price(
                    securities=stocks,
                    start_date=None,
                    end_date=today,
                    fre_step="1d",
                    fields=["open","is_paused","is_st","prev_close","high_limit","low_limit"], # 获取全部数据列，其中昨日收盘价是用来计算总股本的
                    # fq="pre",
                    fq=None, # 这里动态复权或者不复权都是可以的
                    bar_count=1,
                    is_panel=1).to_frame()
                df=df.reset_index()
                df=df[(df["is_paused"]==0)&(df["is_st"]==0)]
                # df=df[df["open"]>4]
                df=df.rename(columns={"minor": "代码","high_limit":"涨停价","low_limit":"跌停价"})
                df=df[["代码","涨停价","跌停价"]]
                stocks=df["代码"].tolist
                
                # 基本面数据【按说应该加上净资产收益率及其增长率】
                df=get_fundamentals(query(
                    asharevalue.symbol,#代码
                    # asharevalue.total_shares,#总股本
                    # # asharevalue.float_shares,#流通股本
                    asharevalue.total_mv,#总市值
                    # income.net_profit,#净利润
                    income.np_atsopc,#归母净利润
                    income.overall_income,#营业总收入
                    (balance.total_liab_and_holders_equity-balance.total_liabilities),#净资产
                    (balance.total_liab_and_holders_equity/balance.total_liabilities),#资产负债率
                    # balance.rd_cost,#开发支出
                    # asharevalue.pe_mrq,#市盈率【影响模型效果】
                    # asharevalue.pb_mrq,#市净率MRQ
                    # asharevalue.ps_ttm,#市销率
                    growth.overall_income_growth_ratio,#营业收入同比增长率
                    growth.net_asset_growth_ratio,#净资产(同比增长率)growth是年度
                    growth.np_atsopc_yoy,#归母净利润(同比增长率)growth
                    # growth_sq.np_atsopc_yoy,#归母净利润(同比增长率)growth_sq是季度
                    # asharevalue.dividend_rate_ttm,#股息率
                ).filter(
                    asharevalue.symbol.in_(stocks),
                    (balance.total_liab_and_holders_equity-balance.total_liabilities)>0,#非资不抵债
                    income.np_atsopc > 0,#净利润大于0
                ),date=yesterday).fillna(0).set_index("asharevalue_symbol")#代码设置成索引
                #设置对应的基本面数据
                baselist=["市值",
                        "归母净利润",
                        "营业总收入",
                        "净资产",
                        "资产负债率", 
                        # "开发支出",#添加开发支出因子会导致大市值股票被认为高估，原因是大市值标的普遍研发占比低
                        "营收增速",
                        "净资产增速",
                        "净利润增速",
                        # "股息率",#这个是新加上的
                        ]
                df.columns=baselist#列名赋值，用前一天的基本面数据同时进行训练和输出
                # print("基本面数据",df)
                for thisfactor in baselist:#直接处理因子避免空值
                    print("当前因子为",thisfactor)
                    df[thisfactor]=signlog(df[thisfactor])
                df=df.reset_index(drop=False)
                df=df.rename(columns={"asharevalue_symbol":"代码"})
                # industry factor行业因子（申万宏源一级行业）
                industrydf=get_industry_relate(types="s_industryid1",date=yesterday)
                print("申万一级行业",industrydf)
                for sector in industrydf.index.tolist():#遍历所有行业
                    tarsector=industrydf[industrydf.index==sector]["industry_symbol"].values[0]
                    istocks=get_industry_stocks(tarsector,yesterday)#获取成分股
                    # print("成分股",istocks,tarsector)
                    df.loc[df["代码"].isin(istocks),sector]=1#使该行业的成分股在该行业名列的值为1
                    df.loc[~(df["代码"].isin(istocks)),sector]=0
                # print("添加行业数据",df)

                # SVR model
                svr=SVR(kernel="rbf")
                # training model #预测的目标是市值【这是一个估值模型】
                df.set_index('代码',inplace=True)#重新设置索引，避免后面处理有问题
                Y=df["市值"]
                X=df.drop("市值", axis=1)
                model=svr.fit(X,Y)#根据前一天的基本面和市值情况预测
                # stocks并选股
                r=Y-pd.Series(svr.predict(X),Y.index,name='估值')#计算的低估程度【负数为低估】
                r=r.reset_index(drop=False)
                r=r.rename(columns={"asharevalue_symbol":"代码"})
                r=pd.DataFrame(r)
                # 估值数据和基本面数据拼接
                df=df.reset_index(drop=False)
                df=df.rename(columns={"asharevalue_symbol":"代码"})
                r=r.merge(df[["市值","代码"]],on="代码")
                r=r.rename(columns={0:'估值'},inplace=False)
                # r["市值（反标准化）"]=reverse_signlog(df["市值"])#对标准化的市值进行反标准化
                # r.to_csv("r.csv")
                r=r[r["估值"]<0]#只选择低估的标的
                print(type(r))#DataFrame格式
                r=r.sort_values(by='估值',ascending=True)#ascending=True从小到大排序
                # r.to_csv("r.csv")
                dftwo=r.nsmallest(math.ceil(1.2*numbuystock),"估值")
                dfone=r.nsmallest(math.ceil(numbuystock),"估值")
                dftwo=dftwo.sort_values(by='估值',ascending=True)#ascending=True从小到大排序
                dfone=dfone.sort_values(by='估值',ascending=True)#ascending=True从小到大排序
                # dftwo.to_csv("dftwo实盘.csv")
                # dfone.to_csv("dfone实盘.csv")
                # #板块推送【标记日为交易日，所有数据用的前一天的】
                # re=custom_sector(str(int(tradeday[index])),'insert',[])#新建一空板块
                # print(re)
                # for stock in dfone["代码"].tolist():
                #     re=custom_sector(str(int(tradeday[index])), 'append', [stock]) # 增加板块1的股票列表
                #     print(re)
                name="估值"
                try:
                    re=custom_sector(str(int(tradeday[index]))+name,'insert',dfone["代码"].tolist())#新建一个板块【直接插入容易导致顺序不对】
                    print(re)
                except:
                    re=custom_sector(str(int(tradeday[index]))+name,'remove') # 删除无用板块
                    print("删除板块返回值",re)
                    re=custom_sector(str(int(tradeday[index]))+name,'insert',dfone["代码"].tolist())#新建一个板块【直接插入容易导致顺序不对】
                    print(re)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 